## A wrapper for OPA within AQUA

In [1]:
from aqua import OPAgenerator

Initialize the OPA generator class, feeding it with the most of the information you need. Which variable, which model source and where to store the data.  Please change the outdir if you want to try this notebook!

In [8]:
model = 'IFS'
exp = 'tco2559-ng5'
source = 'ICMGG_atm2d'
outdir = '/home/b/b382321/scratch/one_pass'
tmpdir = '/home/b/b382321/scratch/one_pass'

In [15]:
opaopa = OPAgenerator(model=model, exp=exp, source=source, var='2t', frequency='daily',
            outdir=outdir, tmpdir=tmpdir, definitive=False, loglevel ='INFO')


2023-05-17 16:13:25 :: opa_generator :: WARNING  -> IMPORTANT: no file will be created, this is a dry run
2023-05-17 16:13:25 :: opa_generator :: WARNING  -> Variable(s) to be processed: 2t
2023-05-17 16:13:25 :: create_folder :: WARNING  -> Folder /home/b/b382321/scratch/one_pass/IFS/tco2559-ng5/daily already exists
2023-05-17 16:13:25 :: create_folder :: WARNING  -> Folder /home/b/b382321/scratch/one_pass already exists


Then you can retrieve the data using the `Reader`. This is based on a streaming generator so far since we are using data from NextGEMS, will be later replaced with direct access from `FDB`. It also gave a look at the data so that it can infer the timedelta

In [16]:
opaopa.retrieve()

2023-05-17 16:13:27 :: opa_generator :: INFO     -> Accessing catalog for IFS-tco2559-ng5-ICMGG_atm2d...
2023-05-17 16:13:27 :: opa_generator :: INFO     -> Getting the timedelta to inform the OPA...
2023-05-17 16:13:28 :: Reader :: INFO     -> tprate: m --> m s**-1
2023-05-17 16:13:28 :: Reader :: INFO     -> tprate: corrected dividing by accumulation time 3600 s
2023-05-17 16:13:28 :: Reader :: INFO     -> Fixing tp to tprate. Unit fix: factor=0.000278, offset=0.000000
2023-05-17 16:13:28 :: Reader :: INFO     -> Grib attributes for cprate: {'paramId': '172143', 'long_name': 'Mean convective precipitation rate', 'units': 'm s**-1', 'cfVarName': 'cprate', 'shortName': 'cprate'}
2023-05-17 16:13:28 :: Reader :: INFO     -> var172143: m --> m s**-1
2023-05-17 16:13:28 :: Reader :: INFO     -> var172143: corrected dividing by accumulation time 3600 s
2023-05-17 16:13:28 :: Reader :: INFO     -> Fixing cp to var172143. Unit fix: factor=0.000278, offset=0.000000
2023-05-17 16:13:28 :: Read

This will launch the OPA computation, if the `definitive` flag is `true` it will take ages and probably break the notebook. Otherwise print only the sequential access to the data from the `Reader`.
Note that to print the sequential access the loglevel should be set to `INFO` or `DEBUG`.

In [17]:
opaopa.generate_opa()

2023-05-17 16:13:53 :: opa_generator :: INFO     -> Dask is disabled...
2023-05-17 16:13:53 :: opa_generator :: WARNING  -> Setting up OPA at daily frequency for variable 2t...
2023-05-17 16:13:53 :: opa_generator :: WARNING  -> Initializing the OPA
2023-05-17 16:13:53 :: opa_generator :: WARNING  -> Initializing the streaming generator...


{'stat': 'mean', 'stat_freq': 'daily', 'output_freq': 'monthly', 'time_step': 60, 'variable': '2t', 'save': True, 'checkpoint': True, 'checkpoint_filepath': '/home/b/b382321/scratch/one_pass', 'out_filepath': '/home/b/b382321/scratch/one_pass/IFS/tco2559-ng5/daily', 'checkpoint_file': '/home/b/b382321/scratch/one_pass/checkpoint_2t_daily_monthly_mean.pkl'}


2023-05-17 16:13:54 :: Reader :: INFO     -> tprate: m --> m s**-1
2023-05-17 16:13:54 :: Reader :: INFO     -> tprate: corrected dividing by accumulation time 3600 s
2023-05-17 16:13:54 :: Reader :: INFO     -> Fixing tp to tprate. Unit fix: factor=0.000278, offset=0.000000
2023-05-17 16:13:54 :: Reader :: INFO     -> Grib attributes for cprate: {'paramId': '172143', 'long_name': 'Mean convective precipitation rate', 'units': 'm s**-1', 'cfVarName': 'cprate', 'shortName': 'cprate'}
2023-05-17 16:13:54 :: Reader :: INFO     -> var172143: m --> m s**-1
2023-05-17 16:13:54 :: Reader :: INFO     -> var172143: corrected dividing by accumulation time 3600 s
2023-05-17 16:13:54 :: Reader :: INFO     -> Fixing cp to var172143. Unit fix: factor=0.000278, offset=0.000000
2023-05-17 16:13:54 :: Reader :: INFO     -> mlsprt: m --> m s**-1
2023-05-17 16:13:54 :: Reader :: INFO     -> mlsprt: corrected dividing by accumulation time 3600 s
2023-05-17 16:13:54 :: Reader :: INFO     -> Fixing lsp to m

KeyboardInterrupt: 

You can then add an entry to the catalog so that you can access these data from somewhere else.

In [18]:
opaopa.create_catalog_entry()

2023-05-17 16:14:45 :: opa_generator :: WARNING  -> Creating catalog entry IFS tco2559-ng5 tmp-opa-daily


In [19]:
from aqua import Reader
reader = Reader(model="IFS", exp="tco2559-ng5", source="tmp-opa-daily",areas=False)

In order to really access and work with data a run with `definitive` set to `True` is needed.

This remove the entries (and in the future also the files), once all the computation have been safely concluded

In [20]:
opaopa.clean()

2023-05-17 16:14:52 :: opa_generator :: WARNING  -> Removing catalog entry IFS tco2559-ng5 tmp-opa-daily
